mnist_cnn 예제에 대한 스터디 내용입니다.

In [1]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

'Trains a simple convnet on the MNIST dataset.\n\nGets to 99.25% test accuracy after 12 epochs\n(there is still a lot of margin for parameter tuning).\n16 seconds per epoch on a GRID K520 GPU.\n'

import 과정

In [2]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

Using Theano backend.


AttributeError: 'module' object has no attribute 'find_graphviz'

batch_size : 한 epoch에서 실행시키는 단위(배치) 크기
nb_classes : 클래스 수
nb_epoch : epoch 수

In [ ]:
BATCH_SIZE = 128
NUM_CLASSES = 2
NUM_EPOCHS = 12

# number of convolutional filters to use
NUM_FILTERS = 32
# size of pooling area for max pooling
NUM_POOL = 2
# convolution kernel size
NUM_CONV = 3

train_img_filename = './tb_data/orgin_64x64_gray_5k_train_img.bin'
train_label_filename = './tb_data/orgin_64x64_gray_5k_train_label.bin'
test_img_filename = './tb_data/orgin_64x64_gray_1k_test_img.bin'
test_label_filename = './tb_data/orgin_64x64_gray_1k_test_label.bin'

TRAIN_DATA_COUNT = 5000
TEST_DATA_COUNT = 1000
VALIDATION_DATA_COUNT = 150

IMG_CHANNELS = 1
IMG_ROWS = 64
IMG_COLS = 64

#### orgin_64x64_color

train_img_filename = 'orgin_64x64_color_5k_train_img.bin'
train_label_filename = 'orgin_64x64_color_5k_train_label.bin'
test_img_filename = 'orgin_64x64_color_1k_test_img.bin'
test_label_filename = 'orgin_64x64_color_1k_test_label.bin'

TRAIN_DATA_COUNT = 5000
TEST_DATA_COUNT = 1000
VALIDATION_DATA_COUNT = 150

IMG_CHANNELS = 3
IMG_ROWS = 64
IMG_COLS = 64

#### rotated_64x64_color

train_img_filename = 'rotated_64x64_color_60k_train_img.bin'
train_label_filename = 'rotated_64x64_color_60k_train_label.bin'
test_img_filename = 'rotated_64x64_color_12k_test_img.bin'
test_label_filename = 'rotated_64x64_color_12k_test_label.bin'

TRAIN_DATA_COUNT = 60000
TEST_DATA_COUNT = 12000
VALIDATION_DATA_COUNT = 1800

IMG_CHANNELS = 3
IMG_ROWS = 64
IMG_COLS = 64

#### rotated_64x64_gray

train_img_filename = 'rotated_64x64_gray_60k_train_img.bin'
train_label_filename = 'rotated_64x64_gray_60k_train_label.bin'
test_img_filename = 'rotated_64x64_gray_12k_test_img.bin'
test_label_filename = 'rotated_64x64_gray_12k_test_label.bin'

TRAIN_DATA_COUNT = 60000
TEST_DATA_COUNT = 12000
VALIDATION_DATA_COUNT = 1800

IMG_CHANNELS = 1
IMG_ROWS = 64
IMG_COLS = 64

In [ ]:
def load_img(filename, count, channel, row, col):
    print('Loading data from', filename)
    
    fp = open(filename, 'rb')
    buf = fp.read(count * channel * row * col)
    data = np.frombuffer(buf, dtype=np.uint8)
    data = data.reshape(count, channel, row, col)

    print('The shape of loaded data is ', data.shape)

    data = data.astype('float32')
    data /= 255

    return data

In [ ]:
def load_labels(filename, count, classes):
    print('Loading labels from ', filename)
    
    fp = open(filename, 'r')
    buf = fp.read(count)
    data_bin = []
    for i in buf:
        data_bin.append(i)
    data = np.asarray(data_bin, dtype=np.uint8, order='C')

    print('The shape of loaded labels is ', data.shape)
    
    # convert class vectors to binary class matrices
    data = np_utils.to_categorical(data, classes)

    return data

In [ ]:
# the data, shuffled and split between train and test sets
train_img = load_img(train_img_filename, TRAIN_DATA_COUNT, IMG_CHANNELS, IMG_ROWS, IMG_COLS)
test_img = load_img(test_img_filename, TEST_DATA_COUNT, IMG_CHANNELS, IMG_ROWS, IMG_COLS)
train_labels = load_labels(train_label_filename, TRAIN_DATA_COUNT, NUM_CLASSES)
test_labels = load_labels(test_label_filename, TEST_DATA_COUNT, NUM_CLASSES)

In [ ]:
validation_img = train_img[:VALIDATION_DATA_COUNT, ...]
validation_labels = train_labels[:VALIDATION_DATA_COUNT, ...]

train_img = train_img[VALIDATION_DATA_COUNT:, ...]
train_labels = train_labels[VALIDATION_DATA_COUNT:, ...]

모델 구축

In [ ]:
model = Sequential()
 
model.add(Convolution2D(NUM_FILTERS, NUM_CONV, NUM_CONV,
                        border_mode='valid',
                        input_shape=(1, IMG_ROWS, IMG_COLS)))
model.add(Activation('relu'))
model.add(Convolution2D(NUM_FILTERS, NUM_CONV, NUM_CONV))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(NUM_POOL, NUM_POOL)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES))
model.add(Activation('softmax'))

compile 및 fit

In [ ]:
from IPython.display import SVG
from keras.utils.visualize_util import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(train_img, 
          train_labels, 
          batch_size=BATCH_SIZE, 
          nb_epoch=NUM_EPOCHS,
          verbose=1, 
          validation_data=(validation_img, validation_labels))

테스트

In [ ]:
score = model.evaluate(test_img, test_labels, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])